University of Helsinki, Master's Programme in Mathematics and Statistics  
MAST32001 Computational Statistics, Autumn 2022  
Luigi Acerbi  

# Home exam

**Deadline:** Friday 28 October at 23:55

- Please return your solutions for Problem 2 as a separate Python source file `problem2.py` and the others in this notebook to Moodle.
- **You must answer Problems 1-3. Please answer 2 out of 3 of the remaining problems (Problems 4-6).**
- Each problem is worth 6 points. The maximum score is 30 points.
- Please note that this exam is *not* teamwork. You must find the answers to the questions just by yourself with no help from others.
- Please write your full name, email address and student ID number in the cell below. By doing so, you are confirming that you are the only author of the answers and solutions in the submitted notebook and code:

## Home exam taker

- **Name:** Victor Manuel Yeom Song
- **Email address:** victor.yeomsong@helsinki.fi
- **Student ID:** 015148944

## General instructions (IMPORTANT!)

1. When returning your solutions, please leave your code and output in the notebook. You will not get points for results that are not included in the notebook even if they could be obtained by running the code. You will not get results if you provide the solutions without the code used to generate them.
2. When returning your solutions, please make sure the notebook can be run cleanly using "Cell" / "Run All".
3. Please make sure there are no dependencies between solutions to different problems.
4. Please make sure that your notebook will not depend on any local files.
5. Please make sure that the solutions for each problem in your notebook will produce the same results when run multiple times, i.e. remember to seed any random number generators you use (e.g., `npr.seed()`!).
6. Please make your code readable. You are not required to write high-quality code, but make some effort to clean it up and add comments when necessary so that it is easy to follow what's going on. (But do not exceed in the opposite direction of explaining every single line.)
7. Make sure you only use the functions specified in the statement of each problem. You can use `matplotlib` plotting in all problems if you wish.
8. The problems are meant to be solved based on contents of the course. You can reuse functions from the class exercises. You should not search for solutions online and in any case must not use any solutions possibly found.
9. **Remember to fill in your full name, email address and student ID in the cell above.** (If for some reason you don't have a student ID yet, leave that empty.)

---

## COMPULSORY PROBLEMS

---

## 1. Check and fix my MCMC (COMPULSORY)

Which of the MCMC algorithms presented below will *eventually* converge to produce samples from the target probability distribution defined by the corresponding `ltarget(x)` functions?

- Justify your answers (both positive and negative) briefly.
- For each sampler that will not converge to the correct distribution, suggest a change to fix it to converge to the intended target distribution.
- You should never change the target distribution. 
- Your fix should keep the function `sample_q(x)` for drawing samples from the proposal the same, if possible. **Only change it if there is no other way to fix the sampler**.

### Problem descriptions

In all cases, we assume we are using a Metropolis-Hastings sampler that takes three arguments: 
- the log-target density `ltarget(x)`;
- the proposal density `eval_logq(xp, x)` evaluating $\log q(x';x)$, i.e. the log probability of proposing $x'$ denoted by `xp` when the current state is $x$; 
- a function `sample_q(x)` to draw samples $x'$ from the proposal $q(x' ; x)$.

We assume that the sampler is initialized at a valid point `x0` (for which the log-target is not `-np.inf`). The sampler will take `2*n` samples, throwing away the first `n` for warmup. `n` is assumed to be a sufficiently large number to ensure convergence *if* the algorithm is correct. For some problems, the required `n` might be astronomically large, but that's fine: this problem is not asking about efficiency, but about *correctness*.

The following snippet of code is used to draw the samples and compute the acceptance probability:
``` {python}
import numpy as np
import numpy.random as npr
import scipy.stats as sps

xp = sample_q(x)
accrate = np.minimum(1, np.exp(ltarget(xp) + eval_logq(x, xp) - ltarget(x) - eval_logq(xp, x)))
```

Many samplers are based on a symmetric proposal with $q(x';x) = q(x;x')$ and define `eval_logq(xp, x) = 0`. This is OK if the proposal really is symmetric, because then `eval_logq(x, xp) == eval_logq(xp, x)`, making the corresponding terms to cancel out in the above acceptance rate.

*Notes and hints*: 
- Since the sampler is a Metropolis-Hastings sampler, what you need to check is mostly two things: (1) whether the log-proposal pdf is implemented correctly, and (2) if the proposal would allow the chain to (eventually) explore the entire target distribution.
- **You are not required to run any code: answering the questions and providing code for the fixed functions is enough.**
- Still, as a way to double-check your hypotheses and answers, sometimes it can be useful for you to write code and run the proposed Metropolis-Hastings samplers on the given problems. Write this test code in a separate file, not here.
- For example, plot a normalized histogram of the samples obtained from the MCMC run overlaid on top of the (normalized) true target pdf.
- However, note that the problem is asking if a sampler will *eventually* converge. If the sampler would converge with an astronomically large `n`, it would still count as converging for the purpose of this question. For some problems, you might be unable to see convergence by simply running the sampler on your computer (unless you can run it for an astronomically large `n`), in which case you will have to think asymptotically. In particular, finite-time diagnostic like $\hat{R}$ are not necessarily informative about *asymptotic* convergence.
- Do **not** submit your test code or test plots (those are only useful checks for you). Below, only write your answer and a brief justification including, if needed, a snippet of code for what needs to be fixed.

### The problems:


Case 1:
``` {python}
    def ltarget1(x):
        if np.abs(x) < np.pi:
            return x*np.cos(x)
        else:
            return -np.inf

    def eval_logq1(xp, x):
        return 0

    def sample_q1(x):
        return npr.uniform(-np.pi/2, np.pi/2)
```

Case 2:
``` {python}
    def ltarget2(x):
        return sps.t.logpdf(x-1, df=5)

    def eval_logq2(xp, x):
        return 0

    def sample_q2(x):
        return x-1 + npr.standard_t(df=1)

```

Case 3:
``` {python}
    def ltarget3(x):
        if np.abs(x)>.99 and np.abs(x) < 3:
            return sps.norm.logpdf(x, 0, 1)
        else:
            return -np.inf

    def eval_logq3(xp, x):
        return 0

    def sample_q3(x):
        return x + npr.triangular(-2,0,2)
```

Case 4:
``` {python}
    def ltarget4(x):
        if np.abs(x) < 10:
            return sps.dgamma.logpdf(x,2)
        elif np.abs(x) > 20 and np.abs(x) < 25:
            return -np.inf
        else:
            return -np.abs(x)

    def eval_logq4(xp, x):
        return 0

    def sample_q4(x):
        return x + npr.uniform(-2, 2)
```
Case 5:
``` {python}
    def ltarget5(x):
        return -np.abs(x+1)

    def eval_logq5(xp, x):
        return sps.norm.logpdf(xp,loc=x,scale=1)

    def sample_q5(x):
        return x + npr.exponential(scale=1e-2) - npr.exponential(scale=1e-2)
```

Case 6:
``` {python}
    def ltarget6(x):
        return np.sin(np.mod(x,2))

    def eval_logq6(xp, x):
        return sps.norm.logpdf(xp,loc=x,scale=1e-1)

    def sample_q6(x):
        return x + npr.normal(0, 3e-2)
```

### Your answer:

Mark if the samplers will eventually converge to a proper target distribution by removing the wrong answer:
1. no
2. yes
3. yes
4. yes
5. yes/no
6. yes/no

Briefly justify your answers and explain how to fix the samplers that would not asymptotically converge to the correct distribution:

Note: In questions 1 through 4, the proposals are all symmetrical, so the samplers match their respective log probability densities.

1. The target distribution has a range defined in $(-\pi, \pi)$, but the proposal only produces samples within the interval $\left[-\frac{\pi}{2}, \frac{\pi}{2}\right)$. This means that the chain would NOT be able to explore the entire target distribution. This can be solved by expanding the range of the proposal:
``` {python}
    def sample_q1(x):
        return npr.uniform(-np.pi, np.pi)
```
Or by centering the new sample around x:
``` {python}
    def sample_q1(x):
        return x + npr.uniform(-np.pi/2, np.pi/2)
```
2. Both the target and the proposal follow Student's t-distribution centered around `x-1`, with the only difference being the degrees of freedom, so the proposal covers the whole range of the target. This means that the chain will converge
3. The target is defined in the range $(-3, -0.99)\cup(0.99, 3)$. The proposal covers a range from `x-2` to `x+2`. So if, for example, the last sample `x` is between 0.99 and 1, it could *possibly* jump over to -1 (since $0.99-2=-1.01$), and then it is more likely that the proposal will sample in the range $(-3, -0.99)$. It can take a long while, but it will eventually converge.
4. The target is defined in the range $(-\infty, -25)\cup(-20, 20)\cup(25, +\infty)$. The proposal is a normal distribution, centered around `x-2` and standard deviation `2`. The normal distribution theoretically covers all the reals, and the range for the target is a subset of the reals, so this sampler should converge eventually.
5. 
6. 


## 2. Numerical computation of probabilities (COMPULSORY)

For each of the cases below, write a function to compute the requested log-probability densities in a numerically stable manner. The functions will be tested on a number of inputs and the scoring depends on the number of cases handled correctly. The functions will only be tested with valid inputs satisfying the constraints defined below.

*Note*: Your code may only use basic `numpy`, `scipy.special`, and `scipy.linalg` functions (including `numpy.linalg`, `scipy.special.gammaln()`, and `scipy.special.multigammaln()`).

**Remember to make sure your code includes necessary imports to run without any additional interaction!**

**Return your solution as a separate file `problem2.py` that contains the definitions of the two functions `lp1()` and `lp2()` as outlined below to Moodle. Your functions should return a single scalar value.**

1. The log-probability density $\log p(x \mid \boldsymbol{\alpha}, \boldsymbol{w})$ of a mixture of $K$ double gamma distributions with
$$p(x \mid \boldsymbol{\alpha}, \boldsymbol{w}) = \sum_{k=1}^K w_k  \mathrm{DG}(x;\; \alpha_k) 
  = \sum_{k=1}^K w_k \frac{1}{2 \Gamma\left(\alpha_k\right)} |x|^{\alpha_k - 1} \exp\left(-|x|\right). $$
Here $0 \le w_i \le 1$, $\sum_{k=1}^K w_k = 1$, $x \in \mathbb R$, and $\alpha_k > 0$.
2. The log-probability density $\log p(\boldsymbol{x} \mid \boldsymbol{V},n)$ of the Wishart distribution with
$$p(\boldsymbol{x} \mid \boldsymbol{V}, n) = \frac{|\boldsymbol{x}|^{(n-p-1)/2}} {2^{np/2} |\boldsymbol{V}|^{n/2} \Gamma_p \left(\frac{n}{2} \right) } \exp\left( \frac{-\text{Tr}(\boldsymbol{V}^{-1}\boldsymbol{x})}{2} \right),$$
where $\boldsymbol{x}, \boldsymbol{V} \in \mathbb R^{p\times p}$ symmetric and positive definite, $n>p-1$ degrees of freedom, $\Gamma_p$ is the multivariate gamma function and $\text{Tr}$ is the matrix trace.

*Notes and hints*:
- To compute $\text{Tr}(\boldsymbol{V}^{-1}\boldsymbol{x})$, one solution is to calculate the explicit inverse $\boldsymbol{V}^{-1}$ via Cholesky decomposition, and then get the trace of matrix product efficiently via [einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html?highlight=einsum#numpy.einsum) without having to compute the diagonal values before the trace. You are welcome to do a more efficient implementation.
- Ensure that you are returning the log probability and not the probability.
- Check that your code is numerically stable and robust to extreme values of the inputs.
- The `problem2.py` file should be a standard Python source file (i.e., text), not a Jupyter notebook.
- Failure to follow the instructions for code submission (e.g., failing to submit the functions as a separate file called `problem2.py`) will result in zero points from this exercise.

In [72]:
# Write your functions lp1() and lp2() to replace the functions here. Do not change the names of the functions.
# Then copy the content of this cell into a separate file named 'problem2.py' to be submitted separately on Moodle.
# The file should include these import instructions and no others.

import numpy as np
import numpy.linalg as lg
import scipy.linalg as slg
import scipy.special as scs

def lp1(x, alpha, w):
    """Returns log p(x | \alpha, w).
    Input:
    x: double
    alpha: np.array, shape: (n,)
    w: np.array, shape: (n,)
    """
    log_denom = np.log(2) + scs.gammaln(alpha) # Compute the log of the denominator of the sum
    log_num = np.log(w) + np.log(np.power(np.abs(x), alpha-1)) - np.abs(x) # log of the numerator of the sum
    return scs.logsumexp(log_num - log_denom) # Exponentiate the log of the sum term, sum over all the terms, and return the log probability

# Helper function to calculate the log-determinant with the Cholesky decomposition
def cholLogDet(A):
    L = lg.cholesky(A)
    return 2*np.sum(np.log(L.diagonal()))

def lp2(x, V, n):
    """Returns log p(x | V, n)
    Input:
    x: np.array, shape: (d,d)
    V: np.array, shape: (d,d)
    n: double, n > d-1
    """
    p = x.shape[0] # Determine p
    
    # Calculate the explicit inverse of L with the Cholesky approach
    inv_chol_L = slg.solve_triangular(lg.cholesky(V), np.eye(p), lower=True)
    inv_V = np.matmul(inv_chol_L.T, inv_chol_L)
    
    # Calculate the log of the numerator and denominator
    log_num = 0.5*(n-p-1) * cholLogDet(x) - 0.5*np.einsum("ij, ij->", inv_V.T, x)
    log_denom = 0.5*n*p*np.log(2) + 0.5*n*cholLogDet(V) + scs.multigammaln(0.5*n, p)
    
    # Return the log probability
    return log_num - log_denom


In [73]:
# Example invocations of the functions, should return a single scalar value

test1 = lp1(.5, np.array([2.,3.]), np.array([0.75, 0.25]))
print(test1)

test2 = lp2(np.eye(2)-.1, np.eye(2)+.1, 3.)
print(test2)

-2.093933725898135
-3.637839915493556


### Your textual answer explaining your solutions:

#### Part 1
To keep numerical stability within the sum, it is best to calculate the log of the terms to avoid issues with double precision. This can be done in a vectorized manner with numpy functions. Then, to avoid over- or underflows in the calculation of the sum, the logsumexp trick is used, which also gives the log probability in this case.

#### Part 2
To keep numerical stability in the calculation of the log probability, we can apply the log first to not calculate the division explicitly, which is prone to errors due to float precision. The log of the numerator and the denominator are calculated.

We need the log of the determinant of a matrix for both $\mathbf{x}$ and $\mathbf{V}$, which are both symmetric and positive definite, so we can use the Cholesky decomposition to calculate it efficiently. This is done in the helper function `cholLogDet()`. This function implements the formula in section 3.2.1 of the course's source material.

To calculate $\text{Tr}(\mathbf{V}^{-1}\mathbf{x})$ we can use the Cholesky approach to calculate the explicit inverse first. We find a matrix $\mathbf{B} \in \mathbb{R}^{p\times p}$ such that $\mathbf{LB}=\mathbf{I}_{p}$, where $\mathbf{L}$ is the Cholesky Decomposition of $\mathbf{V}$. We get the inverse of $\mathbf{L}$, from which we can get $\mathbf{V}^{-1} = \mathbf{L}^{-\intercal}\mathbf{L}^{-1}$. This inverse is calculated efficiently with the solution of a system of linear equations with a triangular matrix. Then, we can calculate the trace with the Einstein summation over the diagonals of $\mathbf{V}^{-1}$ and $\mathbf{x}$.

With this, all that is left is to calculate the log terms on the numerator and the denominator, then return the resulting log probability.

## 3. Approximating an expectation (COMPULSORY)

Develop a Monte Carlo method for approximately evaluating the expectation
$$ V = \mathbb{E}_{p(\theta)}\left[\theta^2\right] = \int_{0}^{\infty} p(\theta) \cdot \theta^2 \,\mathrm{d}\theta $$
over the (normalised) distribution
$$ p(\theta) = \frac{2}{3} \exp\left(-2\sqrt{\theta+1}+2\right) \quad   \text{for} \; \theta \ge 0, \qquad p(\theta) = 0 \quad \text{for} \; \theta < 0.$$

1. You should provide an estimate of $V$ that has an error of at most $\pm 10^{-2}$, but the method must allow increasing the accuracy if needed (typically using more computation).
2. Report both your estimate of $V$ and a 95% confidence interval for the value.
3. Write a brief explanation of the methods used, why you could apply them, and justify why your solution would converge to the correct value given enough computation.

You may use `numpy`, `numpy.random`, `scipy.special`, `scipy.stats`, `matplotlib`, `autograd` and `torch` functions for the task.

*Hints*: 
- You can get a 95% confidence interval by running your procedure $N$ times, reporting the mean as value and using plus/minus two times the standard error of the mean ($\mathrm{SE} = \sigma / \sqrt{N}$) for the interval.
- We saw many Monte Carlo methods in the course that could be used to compute this expectation, such as rejection sampling, importance sampling, various Markov Chain Monte Carlo techniques, etc. Pick the one you think would be most suitable here. There are no extra points for choosing an unnecessarily complex method, if a simple one is sufficient.
- Whichever method you use, remember to fully justify the validity of your solution (depending on the chosen method, this might require diagnostics, referring to the theory, or both).

In [61]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr
import scipy.stats
from scipy.integrate import quad

def log_target(theta):
    return np.log(2) - np.log(3) - 2*np.sqrt(theta + 1) + 2

integ = quad(lambda x: np.exp(log_target(np.array([x])))*x**2, 0, np.inf)

print("Ground truth: ", integ)

npr.seed(0)

N = 20
expectations = np.zeros(N)
beta = 3.5

for i in range(N):
    x_inv_gamma = scipy.stats.invgamma.rvs(1, scale=beta, size=500000)
    is_weights = np.exp(log_target(x_inv_gamma) - scipy.stats.invgamma.logpdf(x_inv_gamma, 1, scale=beta))
    expectations[i] = np.mean(x_inv_gamma**2 * is_weights)

mean_importance = expectations.mean()
standard_error = expectations.std()/np.sqrt(N)

conf_interval = [mean_importance - 2*standard_error, mean_importance + 2*standard_error]

# print(mean_importance)
# print(expectations.std())

Ground truth:  (12.833333333333332, 4.981383311285319e-10)


#### Write your numerical answer below and its 95% confidence interval

In [62]:
v_value = mean_importance # do not hard-code the number: assign the variable used in your previous computations
print('V: {}'.format(v_value))

v_ci = conf_interval # do not hard-code the number: assign the variable used in your previous computations
print('95% confidence interval: {}'.format(v_ci))

V: 12.838384249791465
95% confidence interval: [12.834356082971404, 12.842412416611527]


### Explanation of your solution and justification of its correctness

*Replace this with your explanation*.

---

## OPTIONAL PROBLEMS: CHOOSE 2 OUT OF 3
---

Please note that answering all three problems will not give you extra points.

However, if you decide to return three problems, the problem with the lowest score out of the three will be discarded.

## 4. Resampling for automatic testing of exercise solutions

A teacher of a course in computational statistics is developing exercises for the students. The exercises are checked by an automated system that can be set to accept answers falling to a specified interval $[l, u]$. The exercises involve randomness so the lecturer cannot be certain what values the students will get. The teacher wants to set up the system so that it will accept the solutions obtained by the students after running a given number of iterations with a high probability.

In one specific exercise task, the students are studying the bootstrap confidence intervals of the *skewness* of a set of $N$ numbers $X = \{ x_1, \dots, x_N \}$. For more information about the skewness of a distribution, see [here](https://en.wikipedia.org/wiki/Skewness) (but details are not needed for this problem). 

Roughly speaking, negative skewness indicates that the left tail of a distribution contains more mass than the right one, and vice versa for positive skewness. Finally, zero skewness means that both tails are evenly balanced. It is often used to indicate deviation from Gaussianity, since any symmetric distribution like the normal has zero skewness.

The students are told to use the following (biased) estimator of the skewness:

$$\textrm{Skew}[X] = \frac{\mu_3}{\sigma^3}  = \frac{ \frac{1}{N} \sum_{n=1}^N (x_n - \mu)^3}{\left( \frac{1}{N}\sum_{n=1}^N (x_n - \mu)^2\right)^{3/2}},
\quad \text{where } \mu = \frac{1}{N} \sum_{n=1}^N x_n.$$
The students' task is to draw 1000 bootstrap samples and estimate the central 95% confidence interval of the skewness from the samples.

Your task is to develop a resampling method the teacher could use to accurately estimate central 95% confidence intervals for the end points of the confidence intervals the students will obtain from their simulation. The teacher has limited computational resources and the data set is big, so they can only estimate $\textrm{Skew}$ for at most 10000 bootstrapped sets of the original data similar to the ones the students are using, but can perform futher computation on scalar values computed from these sets.

1. Implement the student bootstrap solution. What is the confidence interval you obtain for the data set loaded below?
2. Independently of your solution above, implement the teacher checker. Remember to respect the limits on computation but try to find the most accurate possible solution within these limits. Test your solution on the data set loaded below and report the confidence intervals obtained for the lower and upper ends of the confidence intervals obtained by the students.

Remember to briefly explain the outline of your solution and to justify why it works.

You may use basic `numpy`, `numpy.random`, `matplotlib` and `pandas` functions for the task.

In [5]:
import pandas as pd
import numpy as np
import numpy.random as npr

p2_data = pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/toydata2.txt', header=None).values[:,0]
print(np.mean(p2_data), np.std(p2_data))

3.9695260800000005 1.4482060147882945


#### Write your numerical answer to part 1 below: what's the 95% confidence interval?

In [6]:
skew_ci = np.nan * np.ones(2) # do not hard-code the number: assign the variable used in your previous computations
print('95% confidence interval (student''s task): {}'.format(skew_ci))

95% confidence interval (students task): [nan nan]


#### Write your numerical answer to part 2 below: what are the 95% confidence intervals for the lower and upper confidence intervals obtained by the students?

In [7]:
skew_lower_ci_ci = np.nan * np.ones(2) # do not hard-code the number: assign the variables used in your previous computations
skew_upper_ci_ci = np.nan * np.ones(2) # do not hard-code the number: assign the variables used in your previous computations

print('95% confidence interval on lower confidence interval: {}'.format(skew_lower_ci_ci))
print('95% confidence interval on upper confidence interval: {}'.format(skew_upper_ci_ci))

95% confidence interval on lower confidence interval: [nan nan]
95% confidence interval on upper confidence interval: [nan nan]


### Your textual answer explaining your solutions:

*Replace this with your answer*.

## 5. MCMC sampling and convergence

Implement a random walk Metropolis-Hastings MCMC sampler for the following model. Tune your sampler to work efficiently, and briefly justify and explain how you did this. **Carefully check the convergence of your sampler with adequate diagnostics** and briefly justify and explain how you did this.

As our model we will consider a logistic regression model fitted to the observations $((\boldsymbol{x}_1, y_1), (\boldsymbol{x}_2, y_2), \dots, (\boldsymbol{x}_N, y_N))$. The code below will load the matrix $\boldsymbol{x}$ (each row is one sample) and vector $\boldsymbol{y}$.

The values $\boldsymbol{x}_1, \dots, \boldsymbol{x}_N$ are the *input values* and $y_1, \dots, y_N$ are the binary *target values*.

The data set comes from the Framingham Heart Study. The target variable (HAS_CHD) indicates whether the subject was diagnosed with heart disease. The input variables are in order:
  1. systolic blood pressure (SBP),
  2. diastolic blood pressure (DBP),
  3. number of cigarettes smoked per day (CIG), and 
  4. age (AGE).

The loading code normalises these to have zero mean and standard deviation $1/2$ to get the regression coefficients to a shared scale.

The logistic regression model is defined with the help of a logistic link function
$$ h_{\beta_0, \boldsymbol{\beta}}(\boldsymbol{x}_n) = \frac{1}{1 + e^{-(\beta_0 + x_{n1} \beta_1 + \dots + x_{n4} \beta_4})}, $$
where $\beta_0$ is the intercept of the model and $\boldsymbol{\beta} = (\beta_1, \dots, \beta_4)$ are the regression coefficients.
The likelihood of the model is
$$ p(\boldsymbol{y} \mid \boldsymbol{x}, \beta_0, \boldsymbol{\beta}) = \prod_{n=1}^N h_{\beta_0,\boldsymbol{\beta}}(\boldsymbol{x}_n)^{y_n} (1 - h_{\beta_0,\boldsymbol{\beta}}(\boldsymbol{x}_n))^{(1-y_n)}. $$

The regression weights $\beta_0$ and $\boldsymbol{\beta}$ have the hierarchical prior
$$ p(\beta_j) = \mathcal{N}(\beta_j;\; 0, \sigma_{\beta}^2), \quad j = 0, 1, \dots, 4, $$
where $\sigma_{\beta}$ is the standard deviation of the prior of $\beta_j$ with the prior 
$$ p(\sigma_{\beta}) = \chi^2(df = 5), $$ 
a chi-squared distribution with degrees of freedom $df=5$.


The full set of model parameters to infer is thus $\theta = (\sigma_{\beta}, \beta_0, \beta_1, \dots, \beta_4)$. Note that $\sigma_{\beta} > 0$, while $\beta_0, \beta_1, \ldots, \beta_4 \in (-\infty, \infty)$.

Write an MCMC sampler to sample from the joint posterior distribution $p(\theta \mid \boldsymbol{x}, \boldsymbol{y}) = p(\sigma_{\beta}, \beta_0, \beta_1, \dots, \beta_4 \mid \boldsymbol{x}, \boldsymbol{y})$ of all model parameters $\theta$.
Compute and print the posterior medians as well as 25% and 75% quantiles of the marginal posteriors of each parameter (also known as interquartile range).
Plot scatter plots with 1000 representative points from the pairwise marginal distributions of all pairs of parameters. Are some pairs of parameters unusually strongly correlated?

You may use `numpy`, `numpy.random`, `pandas`, `scipy.special`, `scipy.stats` and `matplotlib` functions for the task.

*Hint*: You can use `scipy.stats.chi2` for the chi-squared distribution.

In [8]:
%matplotlib inline
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt

heart_data = pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/heart_data.txt', sep='\t')
# Extract input variables
p4_x = heart_data[['SBP', 'DBP', 'CIG', 'AGE']].values.astype(float)
# Extract target variable
p4_y = heart_data['HAS_CHD'].values.astype(float)
# Remove mean from inputs
p4_x -= np.mean(p4_x, 0)
# Standardise input variance
p4_x /= 2*np.std(p4_x, 0)

#### Write your numerical answers below:

In [9]:
posterior_median = np.nan * np.ones(6) # do not hard-code the number: assign the variables used in your previous computations
posterior_lower = np.nan * np.ones(6) # do not hard-code the number: assign the variables used in your previous computations
posterior_upper = np.nan * np.ones(6) # do not hard-code the number: assign the variables used in your previous computations

print('Posterior medians (50% quantiles): {}'.format(posterior_median))
print('Posterior 25% quantiles: {}'.format(posterior_lower))
print('Posterior 75% quantiles: {}'.format(posterior_upper))

Posterior medians (50% quantiles): [nan nan nan nan nan nan]
Posterior 25% quantiles: [nan nan nan nan nan nan]
Posterior 75% quantiles: [nan nan nan nan nan nan]


### Justification/explanation for tuning of the sampler:

*Replace this with your explanation*.

### Justification/explanation for the convergence checking:

*Replace this with your explanation*.

## 6. Variational inference of a multivariate mixture distribution

In this exercise you are asked to use variational inference to approximate a target density $p(\boldsymbol{\theta})$ with a variational approximation $q(\boldsymbol{\theta})$, with $\boldsymbol{\theta} = (\theta_1, \theta_2) \in \mathbb{R}^2$.

The target density is a mixture of $K = 4$ products of [shifted (or non-standardized) Students' t distributions](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html) (each dimension of a mixture component has an independent shifted Student's t) defined as follows:
$$
p(\boldsymbol{\theta}; \boldsymbol{\mu}_1, \ldots, \boldsymbol{\mu}_K, \boldsymbol{\nu}, \boldsymbol{w}) = \sum_{k=1}^K w_k \prod_{d=1}^2 t_{\nu_{k}}\left(\theta_d; \mu_{k,d} \right)
$$
where $\nu_k$ are degrees of freedom for each component, $\mu_{k,d}$ are the location parameters for each shift, and the mixture parameters are the following:
- $\boldsymbol{\mu}_1 = (-4, 2)$, $\boldsymbol{\mu}_2 = (-2, -3.5)$, $\boldsymbol{\mu}_3 = (0, 0)$, $\boldsymbol{\mu}_4 = (3, 4)$
- $\boldsymbol{\nu} = (40,45,3,30)$
- $\boldsymbol{w} = (0.3,0.21,0.08,0.41)$

As variational distribution we will use a product of independent distributions,

$$q(\boldsymbol{\theta}) = q(\theta_1) \cdot q(\theta_2)$$

where  $q(\theta_1)$ and $q(\theta_2)$ are normal distributions. Fit the variational approximation $q(\boldsymbol{\theta})$ to the density $p(\boldsymbol{\theta})$. Please note that the independence assumption above is equivalent to saying that $q(\boldsymbol{\theta})$ is a multivariate normal with diagonal covariance matrix $\boldsymbol{\Sigma}$.

For the best variational solution you find (according to the ELBO), report the means and standard deviations of $q(\theta_1)$ and $q(\theta_2)$ and its ELBO. Describe how you validated your results and discuss your observations.

You may use `numpy`, `numpy.random`, `scipy.special`, `scipy.stats`, `matplotlib` functions for the task, and any linear algebra function you may need (e.g., `numpy.linalg`). You may use `autograd` and `torch`.

*Hint*: Use the formula of the ELBO found in section 0 of the Lecture 11 class tasks.

#### Write your numerical answers below:

In [10]:
vp_means = np.nan * np.ones(2) # do not hard-code the number: assign the variables used in your previous computations
vp_stds = np.nan * np.ones(2) # do not hard-code the number: assign the variables used in your previous computations
best_elbo = np.nan # do not hard-code the number: assign the variables used in your previous computations

print('Variational posterior means: {}'.format(vp_means))
print('Variational posterior standard deviations: {}'.format(vp_stds))
print('ELBO of best solution: {}'.format(best_elbo))

Variational posterior means: [nan nan]
Variational posterior standard deviations: [nan nan]
ELBO of best solution: nan


### Your textual answer explaining your solution:

*Replace this with your answer*.